In [11]:
# Global imports
import os
import numpy as np
import pandas as pd
import imageio
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, LeakyReLU, MaxPooling2D, Flatten, Dense
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import tensorflow_addons as tfa
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from sklearn.model_selection import KFold
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, LeakyReLU, Flatten, Dense
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from tqdm import tqdm
import gc

In [12]:
# import tensorflow as tf
# import librosa
# #print(tf.config.list_physical_devices('GPU'))
# # Desactivar GPU y forzar uso de CPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         #tf.config.experimental.set_memory_growth(gpus[0], True)
#         pass
#     except RuntimeError as e:
#         print(e)

# #tf.config.set_visible_devices([], 'GPU')

In [13]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # TensorFlow 2.x
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found")


1 Physical GPUs, 1 Logical GPUs


# Load dataset


In [14]:
import pickle
def load_data(filename):
    """
    Load data from a pickle file.

    Args:
        filename (str): The path to the pickle file.

    Returns:
        dict: The loaded data dictionary.
    """
    with open(filename, 'rb') as file:
        data_dict_loaded = pickle.load(file)
    return data_dict_loaded

filename = "../Data/D3TEC.pkl"
data_dict_loaded = load_data(filename)
# data_dict_loaded

In [15]:
data_dict_loaded[3]

{'Marca temporal': '2023/10/11 1:57:08 p.\xa0m. GMT-6',
 'PHQ-9 Score': 3,
 'Age': 62,
 'Gender': 'Female',
 'Lugar de Residencia': 'San Pedro, Nuevo León',
 'Lugar de Procedencia': nan,
 'Social Class': 'Upper Class',
 'Institution': 'CAABI',
 'Medicine': 'Sertrialina',
 'Physical Condition': 'Estenosis Espinal',
 'Mental Health Condition': 'Ansiedad',
 'Depression Diagnosis (level)': nan,
 'PHQ-Binary': 0,
 'audios': {'sm': {0: {'file_path': '../D3TEC Dataset/SM-27\\153.wav',
    'waveform': array([-3.4575351e-07, -3.7821010e-06, -4.8473012e-06, ...,
           -7.4757336e-06, -4.6938076e-06, -1.4753023e-06], dtype=float32),
    'sample_rate': 22050,
    'spectrogram': array([[-41.41863 , -35.586212, -42.963806, ..., -50.34841 , -42.829468,
            -59.43257 ],
           [-42.578403, -34.543472, -38.21392 , ..., -37.266647, -38.4431  ,
            -47.42988 ],
           [-45.220688, -38.50419 , -31.796242, ..., -31.12186 , -36.444393,
            -38.83789 ],
           ...,
  

In [16]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, LeakyReLU, Flatten, Dense, Input, MaxPooling2D, Dropout, Resizing, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import KFold
import datetime
import json
from tqdm import tqdm
from PIL import Image
import gc
import psutil


# Deshabilitar XLA compilación
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=0'

# Ruta base para los archivos de espectrogramas
base_path = "F:\\Github\\Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search\\D3T3C"

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast((y_pred < 0.5) & (y_true == 0), tf.float32))
    possible_negatives = tf.reduce_sum(tf.cast(y_true == 0, tf.float32))
    specificity = true_negatives / (possible_negatives + tf.keras.backend.epsilon())
    return specificity

def log_memory_usage(stage):
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"[{stage}] Memory usage: {memory_info.rss / (1024 ** 2):.2f} MB")

In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, LeakyReLU, Flatten, Dense, Input, MaxPooling2D, Dropout, Resizing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold
import datetime
import json
from tqdm import tqdm
from PIL import Image
import gc
import psutil

# Deshabilitar XLA compilación
os.environ['TF_XLA_FLAGS'] = '--tf_xla_auto_jit=0'

# Variable global para rastrear si la función ya se ha llamado
first_call = True

# Ruta base para los archivos de espectrogramas
base_path = "F:\\Github\\Improving-deep-neural-networks-to-identify-mental-disorders-using-Neural-Architecture-Search\\D3T3C"

def specificity(y_true, y_pred):
    true_negatives = tf.reduce_sum(tf.cast((y_pred < 0.5) & (y_true == 0), tf.float32))
    possible_negatives = tf.reduce_sum(tf.cast(y_true == 0, tf.float32))
    specificity = true_negatives / (possible_negatives + tf.keras.backend.epsilon())
    return specificity

def log_memory_usage(stage):
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"[{stage}] Memory usage: {memory_info.rss / (1024 ** 2):.2f} MB")

class KFoldCNNTester:
    def __init__(self, data_dict, filename, batch_size=1, use_dummy_data=False, use_tf_data=True, model_type='SpectroCNN', num_channels=3, recording_device='sm', num_folds_range=[5, 8, 10], use_gender='All', reset_saves=False):
        self.data_dict = data_dict
        self.filename = filename
        self.batch_size = batch_size
        self.mean_acc_per_fold = []
        self.mean_loss_per_fold = []
        self.mean_precision_per_fold = []
        self.mean_recall_per_fold = []
        self.mean_auc_per_fold = []
        self.mean_specificity_per_fold = []
        self.kfold_list = []
        self.state_file = f'{self.filename}_state.json'
        self.keys = list(data_dict.keys())  # Lista de keys del diccionario
        self.use_dummy_data = use_dummy_data
        self.use_tf_data = use_tf_data  # Flag para elegir el método de generación de datos
        self.model_type = model_type  # Flag para elegir el modelo a utilizar
        self.num_channels = num_channels  # Número de canales para las imágenes
        self.recording_device = recording_device  # Tipo de dispositivo de grabación
        self.num_folds_range = num_folds_range  # Array de K-folds a correr
        self.use_gender = use_gender  # Género a utilizar en el K-fold
        self.reset_saves = reset_saves  # Flag para reiniciar los guardados
        
        if self.reset_saves and os.path.exists(self.state_file):
            os.remove(self.state_file)

    def save_state(self, fold_no, epoch_no, num_folds, checkpoint_list, metrics_per_fold):
        state = {'fold_no': fold_no, 'epoch_no': epoch_no, 'num_folds': num_folds, 'checkpoints': checkpoint_list, 'metrics': metrics_per_fold}
        if os.path.exists(self.state_file):
            with open(self.state_file, 'r') as f:
                old_state = json.load(f)
            if 'model_type' not in old_state:
                old_state['model_type'] = {}
            if self.model_type not in old_state['model_type']:
                old_state['model_type'][self.model_type] = {}
            if self.use_gender not in old_state['model_type'][self.model_type]:
                old_state['model_type'][self.model_type][self.use_gender] = {}
            old_state['model_type'][self.model_type][self.use_gender][str(num_folds)] = state
            state = old_state
        else:
            state = {'model_type': {self.model_type: {self.use_gender: {str(num_folds): state}}}}
        with open(self.state_file, 'w') as f:
            json.dump(state, f)

    def load_state(self):
        if os.path.exists(self.state_file):
            with open(self.state_file, 'r') as f:
                state = json.load(f)
            return state
        return None

    def try_load_weights(self, model):
        state = self.load_state()
        checkpoint_list = []
        metrics_per_fold = []
        if state:
            try:
                checkpoints = state["model_type"][self.model_type][self.use_gender][str(self.num_folds_range[0])]["checkpoints"]
                last_checkpoint = checkpoints[-1]  # Tomar el último checkpoint
                model.load_weights(last_checkpoint["path"])
                checkpoint_list = checkpoints
                metrics_per_fold = state["model_type"][self.model_type][self.use_gender][str(self.num_folds_range[0])].get("metrics", [])
                return state, last_checkpoint["fold_no"], last_checkpoint["epoch_no"], last_checkpoint["num_folds"], checkpoint_list, metrics_per_fold
            except Exception as e:
                print(f"Error loading {last_checkpoint['path']}: {e}")
        return None, 1, 0, self.num_folds_range[0], checkpoint_list, metrics_per_fold

    def run_kfold_test(self):
        model = self.define_model()

        state, fold_no, start_epoch, num_folds, checkpoint_list, metrics_per_fold = self.try_load_weights(model)
        print(f"Loaded state: {state}")
        if state is None:
            fold_no, start_epoch, num_folds = 1, 0, self.num_folds_range[0]
            checkpoint_list = []
            metrics_per_fold = {"acc_per_fold": [], "loss_per_fold": [], "precision_per_fold": [], "recall_per_fold": [], "auc_per_fold": [], "specificity_per_fold": []}

        if not os.path.exists('checkpoints'):
            os.makedirs('checkpoints')

        for num_folds in self.num_folds_range:
            if state and num_folds < int(list(state["model_type"][self.model_type][self.use_gender].keys())[0]):
                continue

            kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

            y_array = np.array([self.data_dict[key]['PHQ-Binary'] for key in self.keys])

            for train_indices, test_indices in kfold.split(self.keys, y_array):
                if state and num_folds == state['num_folds'] and fold_no < state['fold_no']:
                    fold_no += 1
                    continue

                # Filtrar por género si es necesario
                if self.use_gender != 'All':
                    train_indices = [i for i in train_indices if self.data_dict[self.keys[i]]['Gender'] == self.use_gender]
                    test_indices = [i for i in test_indices if self.data_dict[self.keys[i]]['Gender'] == self.use_gender]

                # Obtener las keys correspondientes a los índices
                train_keys = [self.keys[i] for i in train_indices]
                test_keys = [self.keys[i] for i in test_indices]

                checkpoint_filepath_1 = f'checkpoints/{self.filename}_{self.model_type}_{self.use_gender}_{self.recording_device}_checkpoint_{num_folds}_fold_{fold_no}_epoch_{start_epoch + 1}.h5'
                checkpoint_filepath_2 = f'checkpoints/{self.filename}_{self.model_type}_{self.use_gender}_{self.recording_device}_checkpoint_{num_folds}_fold_{fold_no}_epoch_{start_epoch + 2}.h5'

                self.try_load_weights(model)

                print('------------------------------------------------------------------------')
                print(f'Training for fold {fold_no} of {num_folds} folds...')

                checkpoint_callback_1 = ModelCheckpoint(filepath=checkpoint_filepath_1, save_weights_only=True,
                                                        monitor='loss', mode='min', save_best_only=False)

                checkpoint_callback_2 = ModelCheckpoint(filepath=checkpoint_filepath_2, save_weights_only=True,
                                                        monitor='loss', mode='min', save_best_only=False)

                callback = EarlyStopping(monitor='loss', patience=3, min_delta=0.0001)

                if self.use_tf_data:
                    train_dataset = self.create_dataset(train_keys)
                    for epoch in range(start_epoch, 100):
                        steps_per_epoch = np.ceil(len(train_keys) / self.batch_size).astype(int)
                        print(f"Training epoch {epoch+1}/100, steps per epoch: {steps_per_epoch}")
                        model.fit(train_dataset, epochs=1, verbose=2, callbacks=[checkpoint_callback_1], steps_per_epoch=steps_per_epoch)
                        if (epoch + 1) % 10 == 0:
                            model.save_weights(checkpoint_filepath_2)
                            checkpoint_list.append({'fold_no': fold_no, 'epoch_no': epoch + 1, 'path': checkpoint_filepath_2})
                        self.save_state(fold_no, epoch + 1, num_folds, checkpoint_list, metrics_per_fold)
                        log_memory_usage(f"Epoch {epoch+1} End")
                        start_epoch = 0
                else:
                    steps_per_epoch = np.ceil(len(train_keys) / self.batch_size).astype(int)
                    for epoch in range(start_epoch, 100):
                        print(f"Training epoch {epoch+1}/100, steps per epoch: {steps_per_epoch}")
                        for i in tqdm(range(0, len(train_keys), self.batch_size), desc=f"Training fold {fold_no}, epoch {epoch+1}"):
                            batch_keys = train_keys[i:i + self.batch_size]
                            X_train, y_train = self.generate_batch_data(batch_keys)
                            model.fit(X_train, y_train, batch_size=self.batch_size, epochs=1, verbose=2, callbacks=[checkpoint_callback_1])
                            del X_train, y_train
                            gc.collect()
                        if (epoch + 1) % 10 == 0:
                            model.save_weights(checkpoint_filepath_2)
                            checkpoint_list.append({'fold_no': fold_no, 'epoch_no': epoch + 1, 'path': checkpoint_filepath_2})
                        self.save_state(fold_no, epoch + 1, num_folds, checkpoint_list, metrics_per_fold)
                        log_memory_usage(f"Epoch {epoch+1} End")
                        start_epoch = 0

                start_epoch = 0

                if self.use_tf_data:
                    test_dataset = self.create_dataset(test_keys)
                    steps_per_epoch = np.ceil(len(test_keys) / self.batch_size).astype(int)
                    print(f"Evaluating fold {fold_no}, steps per epoch: {steps_per_epoch}")
                    scores = model.evaluate(test_dataset, verbose=0, steps=steps_per_epoch)
                else:
                    scores = []
                    for i in tqdm(range(0, len(test_keys), self.batch_size), desc=f"Evaluating fold {fold_no}"):
                        batch_keys = test_keys[i:i + self.batch_size]
                        X_test, y_test = self.generate_batch_data(batch_keys)
                        score = model.evaluate(X_test, y_test, verbose=0)
                        scores.append(score)
                        del X_test, y_test
                        gc.collect()
                    scores = np.mean(scores, axis=0)

                metrics_per_fold["acc_per_fold"].append(scores[1] * 100)
                metrics_per_fold["loss_per_fold"].append(scores[0])
                metrics_per_fold["precision_per_fold"].append(scores[2])
                metrics_per_fold["recall_per_fold"].append(scores[3])
                metrics_per_fold["auc_per_fold"].append(scores[4])
                metrics_per_fold["specificity_per_fold"].append(scores[5])

                print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%; {model.metrics_names[2]}: {scores[2]}; {model.metrics_names[3]}: {scores[3]}; {model.metrics_names[4]}: {scores[4]}; {model.metrics_names[5]}: {scores[5]}')

                if scores[1] * 100 > 60:
                    self.save_model_and_data(model, fold_no, scores, num_folds, train_keys)

                self.save_state(fold_no + 1, 0, num_folds, checkpoint_list, metrics_per_fold)
                fold_no += 1

            self.calculate_means(num_folds, metrics_per_fold)

    def generate_batch_data(self, batch_keys):
        if self.use_dummy_data:
            X = np.random.rand(self.batch_size, 252, 792, self.num_channels)
            y = np.random.randint(0, 2, self.batch_size)
        else:
            X = []
            y = []
            for key in batch_keys:
                info = self.data_dict[key]
                for audio_type, audios in info['audios'].items():
                    for question_number, audio_data in audios.items():
                        if isinstance(question_number, int) and audio_type == self.recording_device:
                            spectrogram_path = base_path.lstrip(".\\") + audio_data['spectrogram_image_path'].lstrip(".\\")
                            spectrogram_image = self.load_image(spectrogram_path)
                            X.append(spectrogram_image)
                            y.append(info['PHQ-Binary'])
            X = np.array(X)
            y = np.array(y)
            X = X / 255.0
        return X, y

    def load_image(self, path):
        img = Image.open(path)
        if self.num_channels == 1:
            img = img.convert('L')
        elif self.num_channels == 2:
            img = img.convert('LA')
        else:
            img = img.convert('RGB')
        
        img_array = np.array(img.resize((792, 252)))
        if self.num_channels == 1 or self.num_channels == 2:
            img_array = np.expand_dims(img_array, axis=-1)

        return img_array

    def define_model(self):
        if self.model_type == 'SpectroCNN':
            return self.define_complete_model()
        elif self.model_type == 'LuisFelipe':
            return self.define_luisfelipe_model()
        else:
            return self.define_reduced_model()

    def define_reduced_model(self):
        model = Sequential([
            BatchNormalization(name='batch_normalization_9'),
            Conv2D(16, kernel_size=(3, 3), padding='same', name='conv2d_6'),
            LeakyReLU(alpha=0.01, name='leaky_re_lu_9'),
            BatchNormalization(name='batch_normalization_10'),
            Conv2D(8, (3, 3), padding='same', name='conv2d_7'),
            LeakyReLU(alpha=0.01, name='leaky_re_lu_10'),
            BatchNormalization(name='batch_normalization_11'),
            Flatten(name='flatten_6'),
            Dense(32, name='dense_6'),
            LeakyReLU(alpha=0.01, name='leaky_re_lu_11'),
            Dense(1, activation='sigmoid', name='dense_7')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC(), specificity])
        return model

    def define_complete_model(self):
        input_shape = (252, 792, self.num_channels)
        inputs = Input(shape=input_shape)

        x = Conv2D(32, kernel_size=(3, 3), padding='same')(inputs)
        x = BatchNormalization()(x)
        x = LeakyReLU(alpha=0.01)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)

        for _ in range(31):
            x = Conv2D(32, kernel_size=(3, 3), padding='same')(x)
            x = BatchNormalization()(x)
            x = LeakyReLU(alpha=0.01)(x)

        x = Flatten()(x)
        x = Dense(256, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(128, activation='relu')(x)
        x = Dropout(0.5)(x)

        outputs = Dense(1, activation='sigmoid')(x)

        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC(), specificity])
        return model

    def define_luisfelipe_model(self):
        model = Sequential()
        model.add(Resizing(128, 10 * time, input_shape=(252, 792, self.num_channels)))
        model.add(Conv2D(30, (3, 3), strides=1, padding="same", activation="relu"))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        model.add(MaxPool2D((2, 2), strides=2, padding="same"))
        model.add(Conv2D(15, (3, 3), strides=1, padding="same", activation="relu"))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        model.add(MaxPool2D((2, 2), strides=2, padding="same"))
        model.add(Flatten())
        model.add(Dense(units=256, activation="relu"))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation="sigmoid"))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC(), specificity])
        return model

    def save_model_and_data(self, model, fold_no, scores, num_folds, train_keys):
        current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        folder_path = f'{self.filename}_{self.model_type}_{self.use_gender}_{self.recording_device}_{current_time}/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        subfolder_path = folder_path + f'{num_folds}-fold_{fold_no}-{scores[1]*100}/'
        if not os.path.exists(subfolder_path):
            os.makedirs(subfolder_path)

        model.save(subfolder_path + f'fold-{fold_no}.h5')

    def calculate_means(self, num_folds, metrics_per_fold):
        mean_acc = sum(metrics_per_fold["acc_per_fold"]) / len(metrics_per_fold["acc_per_fold"]) if metrics_per_fold["acc_per_fold"] else 0
        mean_loss = sum(metrics_per_fold["loss_per_fold"]) / len(metrics_per_fold["loss_per_fold"]) if metrics_per_fold["loss_per_fold"] else 0
        mean_precision = sum(metrics_per_fold["precision_per_fold"]) / len(metrics_per_fold["precision_per_fold"]) if metrics_per_fold["precision_per_fold"] else 0
        mean_recall = sum(metrics_per_fold["recall_per_fold"]) / len(metrics_per_fold["recall_per_fold"]) if metrics_per_fold["recall_per_fold"] else 0
        mean_auc = sum(metrics_per_fold["auc_per_fold"]) / len(metrics_per_fold["auc_per_fold"]) if metrics_per_fold["auc_per_fold"] else 0
        mean_specificity = sum(metrics_per_fold["specificity_per_fold"]) / len(metrics_per_fold["specificity_per_fold"]) if metrics_per_fold["specificity_per_fold"] else 0

        current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        folder_path = f'{self.filename}_{self.model_type}_{self.use_gender}_{self.recording_device}_{current_time}_{num_folds}_mean/'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        with open(folder_path + 'score.txt', 'a+') as file:
            file.write(f'accuracy: {mean_acc}. loss: {mean_loss}. precision: {mean_precision}. recall: {mean_recall}. auc: {mean_auc}. specificity: {mean_specificity}')

    def create_dataset(self, keys):
        def data_generator():
            for key in keys:
                info = self.data_dict[key]
                for audio_type, audios in info['audios'].items():
                    for question_number, audio_data in audios.items():
                        if isinstance(question_number, int) and audio_type == self.recording_device:
                            spectrogram_path = base_path.lstrip(".\\") + audio_data['spectrogram_image_path'].lstrip(".\\")
                            spectrogram_image = self.load_image(spectrogram_path)
                            yield spectrogram_image, info['PHQ-Binary']

        output_signature = (
            tf.TensorSpec(shape=(252, 792, self.num_channels), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int32)
        )

        dataset = tf.data.Dataset.from_generator(
            data_generator,
            output_signature=output_signature
        )
        return dataset.cache().shuffle(buffer_size=1000).batch(self.batch_size).prefetch(tf.data.AUTOTUNE)



In [18]:
# Crear una instancia de la clase con datos dummy
#tester = KFoldCNNTester(data_dict_loaded, 'results_test', batch_size=2, use_dummy_data=True)

# Ejecutar el test K-Fold
#tester.run_kfold_test()


In [19]:
#tester = KFoldCNNTester(data_dict_loaded, 'results', batch_size=1, use_tf_data=True) #32 needs supervision as it stops more

# Run the K-Fold test
#tester.run_kfold_test()


In [20]:
# self, data_dict, filename, batch_size=1, use_dummy_data=False, use_tf_data=True, model_type='SpectroCNN', num_channels=3, recording_device='sm', num_folds_range=[5, 8, 10], use_gender='All'):
tester = KFoldCNNTester(data_dict_loaded, 'results', batch_size=1, use_tf_data=True, model_type='SpectroCNN', num_folds_range=[5, 10], use_gender='Female') #32 needs supervision as it stops more

# Run the K-Fold test
tester.run_kfold_test()


------------------------------------------------------------------------
Training for fold 1 of 5 folds...
Training epoch 1/100, steps per epoch: 30
30/30 - 16s - loss: 181.2409 - accuracy: 0.4333 - precision_1: 0.4615 - recall_1: 0.3750 - auc_1: 0.4509 - specificity: 0.2333 - 16s/epoch - 518ms/step
[Epoch 1 End] Memory usage: 6434.62 MB
Training epoch 2/100, steps per epoch: 30
30/30 - 9s - loss: 58.7218 - accuracy: 0.5333 - precision_1: 0.6429 - recall_1: 0.5000 - auc_1: 0.5579 - specificity: 0.2333 - 9s/epoch - 293ms/step
[Epoch 2 End] Memory usage: 6438.60 MB
Training epoch 3/100, steps per epoch: 30
30/30 - 10s - loss: 68.5319 - accuracy: 0.3333 - precision_1: 0.3478 - recall_1: 0.6154 - auc_1: 0.3484 - specificity: 0.0667 - 10s/epoch - 317ms/step
[Epoch 3 End] Memory usage: 6439.70 MB
Training epoch 4/100, steps per epoch: 30
30/30 - 10s - loss: 26.5292 - accuracy: 0.6333 - precision_1: 0.7500 - recall_1: 0.7143 - auc_1: 0.6085 - specificity: 0.1333 - 10s/epoch - 329ms/step
[Epoc

In [21]:
# self, data_dict, filename, batch_size=1, use_dummy_data=False, use_tf_data=True, model_type='SpectroCNN', num_channels=3, recording_device='sm', num_folds_range=[5, 8, 10], use_gender='All'):
tester = KFoldCNNTester(data_dict_loaded, 'results', batch_size=1, use_tf_data=True, model_type='SpectroCNN', num_folds_range=[5, 10], use_gender='Male') #32 needs supervision as it stops more

# Run the K-Fold test
tester.run_kfold_test()

KeyError: 'Male'